<a href="https://colab.research.google.com/github/GustavoBD-Dev/StatisticalTechniquesAndDataMining/blob/RegressionModels/Prueba_de_Falta_de_Ajuste.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy.stats import f
import warnings
warnings.filterwarnings('ignore')

In [ ]:
x = [150, 150, 150, 200, 200, 200, 250, 250, 250, 300, 300, 300]
y = [77.4, 76.7, 78.2, 84.1, 84.5, 83.7, 88.9, 89.2, 89.7, 94.8, 94.7, 95.9]

data = pd.DataFrame({'x': x, 'y': y})

model = smf.ols(formula='y ~ x', data=data).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.992
Model:                            OLS   Adj. R-squared:                  0.992
Method:                 Least Squares   F-statistic:                     1317.
Date:                Sat, 10 Aug 2024   Prob (F-statistic):           5.99e-12
Time:                        16:49:35   Log-Likelihood:                -10.231
No. Observations:                  12   AIC:                             24.46
Df Residuals:                      10   BIC:                             25.43
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     60.2633      0.744     80.956      0.0

In [ ]:
data['y_pred'] = model.fittedvalues
data['resid']  = data['y'] - data['y_pred']

grouped = data.groupby('x')

pure_error = np.sum([np.sum((group['y'] - group['y'].mean())**2) for name, group in grouped])
pure_error_df = len(data) - len(grouped)

# calcular el error del modelo
model_error = np.sum((data['y'] - data['y_pred'])**2)
model_error_df = len(data) - len(model.params)

# calcular la suma de cuadrados de falta de ajuste
lack_of_fit_ss = model_error - pure_error
lack_of_fit_df = model_error_df - pure_error_df

# caclular la prueba F
ms_lask_of_fit = lack_of_fit_ss / lack_of_fit_df
ms_pure_error  = pure_error / pure_error_df
f_stat = ms_lask_of_fit / ms_pure_error
p_value = f.sf(f_stat, lack_of_fit_df, pure_error_df)

results = pd.DataFrame({
    'Source': ['Lack of fit', 'Pure error'],
    'DF': [lack_of_fit_df, pure_error_df],
    'SS': [lack_of_fit_ss, pure_error],
    'MS': [ms_lask_of_fit, ms_pure_error],
    'F': [f_stat, np.nan],
    'p-value': [p_value, np.nan]
})

print(results)

        Source  DF     SS      MS         F   p-value
0  Lack of fit   2  1.206  0.6030  1.813534  0.224119
1   Pure error   8  2.660  0.3325       NaN       NaN


## Prueba de falta de ajuste


ANOVA de regresión donde el error se desglosa en:
- Error puro
- Falta de ajuste

$$H_0: \text{no hay falta de ajuste}$$

$$H_\alpha : \text{hay falta de ajuste}$$

Estadistico de prueba $F_faltaajuste = 1.813$
$p-vaue = 0.224$

$H_0$ No se rechaza, es decir, no hay falta de ajuste

___